In [1]:
import cv2
# 1. Load bộ phân loại Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") # phát hiện mặt
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml") # phát hiện mắt
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml") # phát hiện miệng

# 2. Load ảnh kính & ria mép (file PNG có nền trong suốt)
glasses = cv2.imread(r"D:\anh\c4.png", cv2.IMREAD_UNCHANGED) # kính mắt
mustache = cv2.imread(r"D:\anh\c5.png", cv2.IMREAD_UNCHANGED) # ria, mũi

def overlay_image(bg, fg, x, y, w, h):
    """Hàm chèn ảnh PNG (có alpha) lên ảnh nền"""
    fg = cv2.resize(fg, (w, h))
    for i in range(fg.shape[0]):
        for j in range(fg.shape[1]):
            if fg[i, j][3] != 0:  # kiểm tra pixel không trong suốt
                if 0 <= y+i < bg.shape[0] and 0 <= x+j < bg.shape[1]:
                    bg[y+i, x+j] = fg[i, j][0:3]
    return bg

# 3. Mở webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Vẽ khung mặt
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)

        # Vùng quan tâm trong khuôn mặt
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Phát hiện mắt -> sẽ chèn kính
       
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        if len(eyes) >= 2:  # cần ít nhất 2 mắt
            # Chèn kính phủ toàn chiều rộng mặt
            frame = overlay_image(frame, glasses, x, y+int(h/5), w, int(h/3))

       
        # Phát hiện miệng -> sẽ chèn ria
        
        mouths = mouth_cascade.detectMultiScale(roi_gray, 1.7, 20) # phát hiện vùng miệng trong khuôn mặt
        for (mx, my, mw, mh) in mouths:
            # Chèn ria (vị trí gần dưới mũi, nên tinh chỉnh lại nếu cần)
            frame = overlay_image(frame, mustache, x+mx, y+my+int(mh/2), mw, int(mh/2))
            break  # chỉ lấy 1 miệng

    cv2.imshow("Face Filter - Haar Cascade", frame)

    # Nhấn q để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

